# Webscrapper for Registro Público de Concesiones

Traditional webscrapping with Beautiful Soup is less useful for aspx websites that expect a user input. This webscraper was built with Selenium intenting to make requests to the form before scraping each page.

Following links are helpful: <br>
Data Source: https://avancedigital.mineco.gob.es/espectro/Paginas/registro-publico-concesiones.aspx <br>
Selenium: https://selenium-python.readthedocs.io/locating-elements.html <br>
Chromedriver: https://chromedriver.chromium.org/home (download necessary)<br>

In [1]:
#pip install requests (for making HTTP requests)
#pip install URLLib3 (URL handling)
#pip install bs4 (in case Selenium couldn’t handle everything)
#pip install selenium (for browser-based navigation)

## Loading libraries and preparing dataframe

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import math

In [3]:
#Create a dataframe to store all of the scraped table data
df = pd.DataFrame(columns = ["Referencia","Titular","NIF/CIF","Domicilio social","Localidad","Provincia","C. Postal",
                             "F. Concesion","F. Caducidad","Susceptible cesion","Susceptible mutualizacion",
                             "Obtenido por transferencia"])

## Making crawler move on the website

In [4]:
# Establish chrome driver and go to report site URL
url = "https://sedeaplicaciones.minetur.gob.es/RPC_Consulta/FrmConsulta.aspx"
driver = webdriver.Chrome('C:/Users/jthoma/Downloads/chromedriver.exe') #use path where chormedriver saved
driver.get(url)

C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\1054195131.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/jthoma/Downloads/chromedriver.exe') #use path where chormedriver saved


#Select Servicio Fijo on Registro Público
serviciofijo_select = driver.find_element(By.XPATH,'//*[@id="MainContent_rblTipoServicio_1"]')
serviciofijo_select.click()

## Scrapping the tables

- 02 ANDALUCÍA
- 03 ARAGÓN
- 04 CANARIAS
- 05 CANTABRIA
- 06 CASTILLA Y LEÓN
- 07 CASTILLA-LA MANCHA
- 08 CATALUÑA
- 09 CIUDAD AUTÓNOMA DE CEUTA
- 10 CIUDAD AUTÓNOMA DE MELILLA
- 11 COMUNIDAD DE MADRID
- 12 COMUNIDAD FORAL DE NAVARRA
- 13 COMUNITAT VALENCIANA
- 14 EXTREMADURA
- 15 GALICIA
- 16 ILLES BALEARS
- 17 LA RIOJA
- 18 PAÍS VASCO
- 19 PRINCIPADO DE ASTURIAS
- 20 REGIÓN DE MURCIA

In [5]:
for comunidad in range(2,21):
    
    #Select Servicio Fijo on Registro Público
    serviciofijo_select = driver.find_element(By.XPATH,'//*[@id="MainContent_rblTipoServicio_1"]')
    serviciofijo_select.click()
    
    #Select Comunidad on Registro Público
    comunidad_select = driver.find_element(By.XPATH,'//*[@id="MainContent_cmbComunidad"]/option['+str(comunidad)+']')
    comunidad_select.click()

    #Click on Buscar
    buscar_button = driver.find_element(By.XPATH,'//*[@id="MainContent_btnBuscar"]')
    buscar_button.click()
    driver.implicitly_wait(5) #Wait 5 seconds for the load
        
    #Getting the number of pages to scrap
    number_concesiones = driver.find_element(By.XPATH,'//*[@id="MainContent_lblTotal"]')
    pages = math.ceil((int(str.split(number_concesiones.text)[0]))/10)

    
    #Flip through all of the records and save them
    for n in range(2, pages+1):
        for i in range(3):
            try:
                mytable = driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]')
                #Read in all of the data into the dataframe
                for row in mytable.find_elements(By.CSS_SELECTOR,'tr'):
                    row_list = []
                    #Add to dataframe accordingly
                    for cell in row.find_elements(By.CSS_SELECTOR,'td'):
                        cell_reading = cell.text
                        if cell_reading == "":
                          checkbox = cell.find_element(By.CLASS_NAME, 'aspNetDisabled').find_element(By.CSS_SELECTOR,'input')
                          cell_reading = checkbox.get_attribute('checked')
                        row_list.append(cell_reading)
                    #Add the list as a row, if possible 
                    try:
                        a_series = pd.Series(row_list, index = df.columns)
                        df = df.append(a_series, ignore_index=True)
                    except:
                        print("Could not append: " + str(row_list))
                break
            except:
                driver.implicitly_wait(5)
        if n%10 == 1:
            #Click second "..." if on greater than page 10
            if n < 20:
                driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[0].click()  
            else:
                driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a')[0].click()  
        else:
            driver.find_element(By.XPATH,"//td/a[text()='" + str(n)+ "']").click()    
        #Wait three seconds so the website doesn't crash
        driver.implicitly_wait(3)
        
    
    #go back to selecting comunidad
    nueva_busqueda = driver.find_element(By.XPATH,'//*[@id="MainContent_btnNuevaBusqueda"]')
    nueva_busqueda.click()
    driver.implicitly_wait(5) #Wait 5 seconds for the load

C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_series = pd.Series(row_list, index = df.columns)


Could not append: []


C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jth

Could not append: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '...']


C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_series = pd.Series(row_list, index = df.columns)


...

C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_series = pd.Series(row_list, index = df.columns)


Could not append: []


C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_6852\2457786483.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jth

Could not append: ['...', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="MainContent_lblTotal"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00466463+2188387]
	Ordinal0 [0x003FE461+1762401]
	Ordinal0 [0x00313D78+802168]
	Ordinal0 [0x00341880+989312]
	Ordinal0 [0x00341B1B+989979]
	Ordinal0 [0x0036E912+1173778]
	Ordinal0 [0x0035C824+1099812]
	Ordinal0 [0x0036CC22+1166370]
	Ordinal0 [0x0035C5F6+1099254]
	Ordinal0 [0x00336BE0+945120]
	Ordinal0 [0x00337AD6+948950]
	GetHandleVerifier [0x007071F2+2712546]
	GetHandleVerifier [0x006F886D+2652765]
	GetHandleVerifier [0x004F002A+520730]
	GetHandleVerifier [0x004EEE06+516086]
	Ordinal0 [0x0040468B+1787531]
	Ordinal0 [0x00408E88+1805960]
	Ordinal0 [0x00408F75+1806197]
	Ordinal0 [0x00411DF1+1842673]
	BaseThreadInitThunk [0x7509FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77137A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77137A6E+238]


#Click on Buscar
buscar_button = driver.find_element(By.XPATH,'//*[@id="MainContent_btnBuscar"]')
buscar_button.click()

driver.implicitly_wait(10) #Wait 10 seconds for the load

## Scrapping the tables

import pandas as pd
import math


#Create a dataframe to store all of the scraped table data
df = pd.DataFrame(columns = ["Referencia","Titular","NIF/CIF","Domicilio social","Localidad","Provincia","C. Postal",
                             "F. Concesion","F. Caducidad","Susceptible cesion","Susceptible mutualizacion",
                             "Obtenido por transferencia"])

#Getting the number of pages to scrap
number_concesiones = driver.find_element(By.XPATH,'//*[@id="MainContent_lblTotal"]')
pages = math.ceil((int(str.split(number_concesiones.text)[0]))/10)

#Flip through all of the records and save them
for n in range(2, pages+1):
    for i in range(3):
        try:
            mytable = driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]')
            #Read in all of the data into the dataframe
            for row in mytable.find_elements(By.CSS_SELECTOR,'tr'):
                row_list = []
                #Add to dataframe accordingly
                for cell in row.find_elements(By.CSS_SELECTOR,'td'):
                    cell_reading = cell.text
                    if cell_reading == "":
                      checkbox = cell.find_element(By.CLASS_NAME, 'aspNetDisabled').find_element(By.CSS_SELECTOR,'input')
                      cell_reading = checkbox.get_attribute('checked')
                    row_list.append(cell_reading)
                #Add the list as a row, if possible 
                try:
                    a_series = pd.Series(row_list, index = df.columns)
                    df = df.append(a_series, ignore_index=True)
                except:
                    print("Could not append: " + str(row_list))
            break
        except:
            driver.implicitly_wait(5)
    if n%10 == 1:
        #Click second "..." if on greater than page 10
        if n < 20:
            driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[0].click()  
        else:
            driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a')[0].click()  
    else:
        driver.find_element(By.XPATH,"//td/a[text()='" + str(n)+ "']").click()    
    #Wait three seconds so the website doesn't crash
    driver.implicitly_wait(3)

In [ ]:
df

In [ ]:
#Write to a csv
df.to_csv("RegistroPublicoConcesiones_General.csv", index= False)